In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
from sklearn.metrics import roc_curve
import pandas as pd
# from apex import amp  # for mixed precision training
from torchvision import models

In [2]:
# Define transforms for data augmentation and normalization
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [3]:
# Set the paths to your training and validation directories
train_dir = '/kaggle/input/morph-splitted/train'
val_dir = '/kaggle/input/morph-splitted/val'
test_dir = '/kaggle/input/morph-splitted/test'

In [4]:
image_datasets = {
    'train': datasets.ImageFolder(train_dir, data_transforms['train']),
    'val': datasets.ImageFolder(val_dir, data_transforms['val']),
    'test': datasets.ImageFolder(test_dir, data_transforms['test'])
}


In [5]:
# Create data loaders
dataloaders = {
    'train': DataLoader(image_datasets['train'], batch_size=16, shuffle=True, num_workers=4),
    'val': DataLoader(image_datasets['val'], batch_size=16, shuffle=False, num_workers=4),
    'test': DataLoader(image_datasets['test'], batch_size=16, shuffle=False, num_workers=4)
}


In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [10]:
# Define ResNet-101 model for binary classification
class ResNet101Binary(nn.Module):
    def __init__(self):
        super(ResNet101Binary, self).__init__()
        self.model = models.resnet101(pretrained=True)
        num_ftrs = self.model.fc.in_features
        self.model.fc = nn.Linear(num_ftrs, 1)  # Output layer with 1 neuron for binary classification
    
    def forward(self, x):
        return self.model(x)

# Initialize the model
model = ResNet101Binary().to(device)
criterion = nn.BCEWithLogitsLoss()  # Binary cross-entropy loss
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Update optimizer to optimize all parameters of the model


In [11]:
num_epochs = 10
best_model_wts = model.state_dict()
best_acc = 0.0
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}


In [13]:
# Training loop
for epoch in range(num_epochs):
    print(f'Epoch {epoch+1}/{num_epochs}')
    print('-' * 10)

    # Each epoch has a training and validation phase
    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()  # Set model to training mode
        else:
            model.eval()  # Set model to evaluate mode

        running_loss = 0.0
        running_corrects = 0

        # Iterate over data with progress bar
        with tqdm(total=len(dataloaders[phase]), desc=f'{phase} Phase', unit='batch') as pbar:
            for i, (inputs, labels) in enumerate(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward pass
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    preds = torch.sigmoid(outputs).round()
                    # Convert labels to match the output size of the model
                    labels = labels.unsqueeze(1).float()  # Convert to shape (batch_size, 1)
#                     print('Output shape:', outputs.shape)
#                     print('Target shape:', labels.shape)
                    loss = criterion(outputs, labels)

                    # Backward pass and optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels)

                # Update progress bar
                pbar.update(1)
                pbar.set_postfix(loss=running_loss / ((pbar.n + 1) * inputs.size(0)),
                                 accuracy=running_corrects.double() / ((pbar.n + 1) * inputs.size(0)))

        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = running_corrects.double() / dataset_sizes[phase]

        print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

        # Deep copy the model
        if phase == 'val' and epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = model.state_dict()

    print()

# Load best model weights
model.load_state_dict(best_model_wts)
print('Best val Acc: {:4f}'.format(best_acc))


Epoch 1/10
----------


train Phase: 100%|██████████| 1500/1500 [04:00<00:00,  6.25batch/s, accuracy=tensor(0.9555, device='cuda:0', dtype=torch.float64), loss=0.115]


train Loss: 0.1148 Acc: 0.9561


val Phase: 100%|██████████| 500/500 [00:27<00:00, 18.45batch/s, accuracy=tensor(0.9844, device='cuda:0', dtype=torch.float64), loss=0.0429]  


val Loss: 0.0430 Acc: 0.9864

Epoch 2/10
----------


...

val Loss: 0.0180 Acc: 0.9942

Epoch 10/10
----------


train Phase: 100%|██████████| 1500/1500 [03:58<00:00,  6.29batch/s, accuracy=tensor(0.9911, device='cuda:0', dtype=torch.float64), loss=0.0238]


train Loss: 0.0238 Acc: 0.9918


val Phase: 100%|██████████| 500/500 [00:26<00:00, 18.80batch/s, accuracy=tensor(0.9794, device='cuda:0', dtype=torch.float64), loss=0.0477]  


val Loss: 0.0478 Acc: 0.9814

Best val Acc: 0.994250


In [14]:
# Test the model
model.eval()
running_loss = 0.0
running_corrects = 0

with torch.no_grad():
    for inputs, labels in dataloaders['test']:
        inputs = inputs.to(device)
        labels = labels.float().view(-1, 1).to(device)

        outputs = model(inputs)
        preds = torch.sigmoid(outputs) > 0.5
        loss = criterion(outputs, labels)

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

test_loss = running_loss / len(image_datasets['test'])
test_acc = running_corrects.double() / len(image_datasets['test'])

print(f'Test Loss: {test_loss:.4f} Acc: {test_acc:.4f}')


Test Loss: 0.0435 Acc: 0.9824


In [16]:
import torch
from torch.utils.data import DataLoader

# Function to create data loaders
def create_data_loader(data_dir, transform, batch_size):
    dataset = datasets.ImageFolder(data_dir, transform)
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=4)
    return data_loader

# Define the transforms for the datasets
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

batch_size = 32

# Create data loaders for each dataset
fm_loader = create_data_loader('/kaggle/input/mad-benchmark/FaceMorpher', transform, batch_size)
mg1_loader = create_data_loader('/kaggle/input/mad-benchmark/MIPGAN_I', transform, batch_size)
mg2_loader = create_data_loader('/kaggle/input/mad-benchmark/MIPGAN_II', transform, batch_size)
oc_loader = create_data_loader('/kaggle/input/mad-benchmark/OpenCV', transform, batch_size)
wm_loader = create_data_loader('/kaggle/input/mad-benchmark/Webmorph', transform, batch_size)


In [17]:
data_loaders = [fm_loader, mg1_loader, mg2_loader, oc_loader, wm_loader]
results = []
losses = []

# Evaluate the model on each dataset
criterion = nn.BCEWithLogitsLoss()
model.eval()

ResNet101Binary(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
    

In [18]:
for loader_idx, data_loader in enumerate(data_loaders):
    running_loss = 0.0
    running_corrects = 0
    total_samples = 0

    dataset_name = ["FaceMorpher", "MIPGAN_I", "MIPGAN_II", "OpenCV", "Webmorph"][loader_idx]
    print(f"Evaluating dataset: {dataset_name}")

    for inputs, labels in tqdm(data_loader, desc=f"Processing {dataset_name}", leave=False):
        inputs = inputs.to(device)
        labels = labels.float().view(-1, 1).to(device)

        outputs = model(inputs)
        preds = torch.sigmoid(outputs) > 0.5
        loss = criterion(outputs, labels)

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        total_samples += inputs.size(0)

    test_loss = running_loss / total_samples
    test_accuracy = running_corrects.double() / total_samples

    results.append(test_accuracy.item())
    losses.append(test_loss)

    print(f"{dataset_name} - Loss: {test_loss:.4f} Acc: {test_accuracy:.4f}")

# Print the final results
print("\nFinal Results:")
names = ["FaceMorpher", "MIPGAN_I", "MIPGAN_II", "OpenCV", "Webmorph"]
for name, accuracy in zip(names, results):
    print(f"{name}: {accuracy:.4f}")


Evaluating dataset: FaceMorpher


FaceMorpher - Loss: 0.6723 Acc: 0.7807
Evaluating dataset: MIPGAN_I


MIPGAN_I - Loss: 0.5266 Acc: 0.8007
Evaluating dataset: MIPGAN_II


MIPGAN_II - Loss: 0.5023 Acc: 0.8238
Evaluating dataset: OpenCV


OpenCV - Loss: 4.5039 Acc: 0.1860
Evaluating dataset: Webmorph


Webmorph - Loss: 5.3649 Acc: 0.2912

Final Results:
FaceMorpher: 0.7807
MIPGAN_I: 0.8007
MIPGAN_II: 0.8238
OpenCV: 0.1860
Webmorph: 0.2912


### **Evaluation Metrics**

In [19]:
import torch
import numpy as np
import pandas as pd
from sklearn.metrics import roc_curve

def calculate_apcer(true_labels, predictions, fixed_bpcer):
    """Calculate APCER at a fixed BPCER."""
    fpr, tpr, thresholds = roc_curve(true_labels, predictions, pos_label=1)
    fpr_target = fixed_bpcer
    closest_fpr_index = np.argmin(np.abs(fpr - fpr_target))
    apcer = 1 - tpr[closest_fpr_index]
    return apcer

def calculate_bpcer(true_labels, predictions, fixed_apcer):
    """Calculate BPCER at a fixed APCER."""
    fpr, tpr, thresholds = roc_curve(true_labels, predictions, pos_label=1)
    tpr_target = 1 - fixed_apcer
    closest_tpr_index = np.argmin(np.abs(tpr - tpr_target))
    bpcer = fpr[closest_tpr_index]
    return bpcer

def calculate_eer(true_labels, predictions):
    """Calculate EER."""
    fpr, tpr, thresholds = roc_curve(true_labels, predictions, pos_label=1)
    frr = 1 - tpr
    eer_index = np.argmin(np.abs(fpr - frr))
    eer = fpr[eer_index]
    return eer

# Define datasets and model predictions
datasets = [fm_loader, mg1_loader, mg2_loader, oc_loader, wm_loader]
names = ["FaceMorpher", "MIPGAN_I", "MIPGAN_II", "OpenCV", "Webmorph"]
fixed_bpcer_values = [0.01, 0.1, 0.2]
fixed_apcer_values = [0.01, 0.1, 0.2]
all_results = []

# Move the model to the GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()  # Ensure the model is in evaluation mode

# Iterate over each dataset
for dataset, name in zip(datasets, names):
    print(f"Evaluating model on dataset: {name}")
    
    # Predictions and true labels
    all_predictions = []
    all_true_labels = []
    for inputs, labels in dataset:
        inputs, labels = inputs.to(device), labels.to(device)  # Move data to the GPU
        with torch.no_grad():  # Disable gradient computation
            predictions = model(inputs)
        all_predictions.append(predictions.detach().cpu().numpy())
        all_true_labels.append(labels.cpu().numpy())
    predictions = np.concatenate(all_predictions)
    true_labels = np.concatenate(all_true_labels)

    # Calculate metrics for each fixed BPCER
    for fixed_bpcer in fixed_bpcer_values:
        print(f"Calculating metrics for fixed BPCER: {fixed_bpcer}")
        apcer = calculate_apcer(true_labels, predictions, fixed_bpcer)
        result = {
            "Dataset": name,
            "Fixed BPCER": f"{fixed_bpcer * 100:.1f}%",
            "APCER": apcer
        }
        all_results.append(result)
    
    # Calculate metrics for each fixed APCER
    for fixed_apcer in fixed_apcer_values:
        print(f"Calculating metrics for fixed APCER: {fixed_apcer}")
        bpcer = calculate_bpcer(true_labels, predictions, fixed_apcer)
        result = {
            "Dataset": name,
            "Fixed APCER": f"{fixed_apcer * 100:.1f}%",
            "BPCER": bpcer
        }
        all_results.append(result)

    # Calculate EER
    eer = calculate_eer(true_labels, predictions)
    result = {
        "Dataset": name,
        "EER": eer
    }
    all_results.append(result)

# Convert the results to a Pandas DataFrame
df_results = pd.DataFrame(all_results)

# Display the DataFrame
print(df_results)


Evaluating model on dataset: FaceMorpher
Calculating metrics for fixed BPCER: 0.01
Calculating metrics for fixed BPCER: 0.1
Calculating metrics for fixed BPCER: 0.2
Calculating metrics for fixed APCER: 0.01
Calculating metrics for fixed APCER: 0.1
Calculating metrics for fixed APCER: 0.2
Evaluating model on dataset: MIPGAN_I
Calculating metrics for fixed BPCER: 0.01
Calculating metrics for fixed BPCER: 0.1
Calculating metrics for fixed BPCER: 0.2
Calculating metrics for fixed APCER: 0.01
Calculating metrics for fixed APCER: 0.1
Calculating metrics for fixed APCER: 0.2
Evaluating model on dataset: MIPGAN_II
Calculating metrics for fixed BPCER: 0.01
Calculating metrics for fixed BPCER: 0.1
Calculating metrics for fixed BPCER: 0.2
Calculating metrics for fixed APCER: 0.01
Calculating metrics for fixed APCER: 0.1
Calculating metrics for fixed APCER: 0.2
Evaluating model on dataset: OpenCV
Calculating metrics for fixed BPCER: 0.01
Calculating metrics for fixed BPCER: 0.1
Calculating metrics